In [1]:
import numpy as np
import plotly.express as px
from sklearn.utils import check_random_state
from PIL import Image, ImageDraw, ImageFont
import os

In [2]:
class PIfinder:
    def __init__(
        self, 
        r,
        seeds,
        ):
        self.seed_x, self.seed_y = check_random_state(seeds[0]), check_random_state(seeds[1])
        self.r = r
        
    def _get_cordinate_ranges(
        self,
        max_samples
    ):
        x = self.seed_x.uniform(-self.r, self.r, max_samples)
        y = self.seed_y.uniform(-self.r, self.r, max_samples)
        return x, y
    
    def _indicator_function(
        self,
        x,
        y
    ):
        return x**2 + y**2 <= self.r**2
    
    def _do_monteCarlo_integration(
        self,
        max_samples
    ):
        constant = 4 * (self.r**2) * (1/max_samples)
        x, y = self._get_cordinate_ranges(max_samples)
        I = constant * np.sum(self._indicator_function(x, y), axis=0)
        return I/self.r**2

In [3]:
pi_object = PIfinder(r=1, seeds=(5, 10))
max_samples = 3000
pi_value = pi_object._do_monteCarlo_integration(max_samples)
pi_value

3.1533333333333333

In [4]:
pi_history = [pi_object._do_monteCarlo_integration(samples) for samples in range(1, max_samples+1)]

In [5]:
r = pi_object.r
max_samples = 3000
x, y = pi_object._get_cordinate_ranges(max_samples)
for samples in range(1, max_samples+1):    
    fig = px.scatter(
        x = x[:samples],
        y = y[:samples],
        color=pi_object._indicator_function(x[:samples], y[:samples]),
        color_discrete_map={0: 'red', 1: 'blue'},
        )
    fig.update_xaxes(showline=False, showgrid=False)
    fig.update_yaxes(showline=False, showgrid=False)
    fig.update_layout(width=600, height=600, showlegend=False,
                      xaxis = dict(range=[-r,r]),
                      yaxis = dict(range=[-r,r]),
                      template='plotly_dark',
                      )
    fig.add_shape(
        type='circle',
        xref='x',
        yref='y',
        x0=-r, 
        y0=-r,  
        x1=r,  
        y1=r,  
        line=dict(color='white', width=0.5),
        )
    fig.update_traces(marker_size=4)
    fig.write_image(f'path/{samples}.png')

In [5]:
def create_animation(image_folder, output_path, frame_duration=4, loop=True):
    images = []
    font_size = 15
    for i in range(1, max_samples+1):
        filename = f'{i}.png'
        image_path = os.path.join(image_folder, filename)
        img = Image.open(image_path)
        draw = ImageDraw.Draw(img)
        font = ImageFont.truetype("/usr/share/fonts/truetype/freefont/FreeMono.ttf", size=font_size)
        draw.text((20, 15), f'Samples: {i}/{max_samples} \nπ: {round(pi_history[i-1],4)}', fill='white', font=font)
        images.append(img)
    images[0].save(output_path, save_all=True, append_images=images[1:], duration=frame_duration, loop=0 if loop else 1)

if __name__ == "__main__":
    folder_path = "path"
    output_gif_path = "path/pi2.gif"
    frame_duration_ms = 4
    loop_animation = True
    create_animation(folder_path, output_gif_path, frame_duration_ms, loop_animation)